### log probs from logits

- 定义
    - (model) logits: $z$, true label: $y$
    - 标准的交叉熵损失，PyTorch 中 reduction='None' 计算的是真实标签 $y$ 对应的负对数概率：

$$
\text{CrossEntropyLoss}(z,y)=-\log p(y|z)=-\log\left(\frac{\exp(z_y)}{\sum_i\exp(z_i)}\right)
$$

- log p = -crossentropyloss

$$
\log p(y|z)=\log\left(\frac{\exp(z_y)}{\sum_i\exp(z_i)}\right)=-\text{CrossEntropyLoss}(z,y)
$$

- log p_y = z_y - logsumexp(z)
    - softmax: $z=(z_1,z_2,\cdots,z_k)$ => $p=(p_1, p_2, \cdots, p_k)$
        - $p_j=\frac{\exp(z_j)}{\sum_i\exp(z_i)}$
    - log-softmax
        - $\log p_j=\log \frac{\exp(z_j)}{\sum_i\exp(z_i)}=\log \exp(z_j) - \log{\sum_i\exp(z_i)}=z_j-\log{\sum_i\exp(z_i)}$
        - $\log p_j=z_j-\text{logsumexp}(z)$
    - $z_y$ 通过 torch.gather 实现；
    - logsumexp 有专门的数值稳定性计算优化
        - $\text{logsumexp}(z)=z_{max}+\log(\sum_j\exp(z_j-z_{max}))$
            - 最大的 z 对应的 exp 项就变成了 $\exp(0)=1$，避免了 overflow；
    - 内存的角度
        - log softmax：`[bsz, seq_len, vocab_size]`
        - z_y - logsumexp(z):
            - z_y: `[bsz, seq_len]`
            - logsumexp(z): `[bsz, seq_len]`

### entropy from logits

> 虽然希望entropy相对高（有一定多样性）但不希望爆炸高（出现乱码）。

- (trl) `ppo_trainer.py`


```python
# logits.shape: (total_tokens, vocab_size)
def entropy_from_logits(logits: torch.Tensor):
    """Calculate entropy from logits."""
    pd = torch.nn.functional.softmax(logits, dim=-1)
    entropy = torch.logsumexp(logits, dim=-1) - torch.sum(pd * logits, dim=-1)
    return entropy

# return: (total_tokens, ), token 级别的熵
```

$$
H=-\sum_vp(v)\log p(v)
$$

- 在 llm 中，就是 generation 生成序列的每个位置 $\pi_\theta(\cdot |q, o_{\lt t})$ 都对应一个词表维度的概率分布

$$
p(v)=\frac{\exp(\text{logits}_v)}{\sum_{v'}\exp(\text{logits}_{v'})}=\frac{\exp(\text{logits}_v)}{Z}
$$

- 则有

$$
\log p(v)=\text{logits}_v-\log Z
$$

- 进一步

$$
H=-\sum_v p(v)\log p(v)=-\sum_v p(v)(\text{logits}_v-\log Z)=\log Z - \sum_v p(v)\text{logits}_v
$$